In [1]:
from importlib import reload
import numpy as np
import time

from ics.cobraCharmer import pfi as pfiControl
reload(pfiControl)

<module 'ics.cobraCharmer.pfi' from '/home/cwen/mhs/devel/ics_cobraCharmer/python/ics/cobraCharmer/pfi.py'>

In [13]:
# pfiCobras = PFI.allocateCobraRange(range(1,43))

mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,1)])
twoCobras = pfiControl.PFI.allocateCobraList([(1,1), (1,2)])

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

# partition module 1 cobras into odd and even sets
moduleCobras2 = {}
for group in 1,2:
    cm = range(group,58,2)
    mod = [1]*len(cm)
    moduleCobras2[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
oddCobras = moduleCobras2[1]
evenCobras = moduleCobras2[2]

In [6]:
#pfi = pfiControl.PFI(fpgaHost='128.149.77.24') #'fpga' for real device.
pfi = pfiControl.PFI(fpgaHost='localhost')
pfi.setFreq(allCobras)

In [14]:
print([f"{c.module},{c.cobraNum}" for c in oddCobras])

['1,1', '1,3', '1,5', '1,7', '1,9', '1,11', '1,13', '1,15', '1,17', '1,19', '1,21', '1,23', '1,25', '1,27', '1,29', '1,31', '1,33', '1,35', '1,37', '1,39', '1,41', '1,43', '1,45', '1,47', '1,49', '1,51', '1,53', '1,55', '1,57']


In [9]:
for cobraGroup in moduleCobras.values():
    pfi.homePhi(cobraGroup, dir='ccw')

In [10]:
pfi.homeTheta(allCobras, dir='ccw')

In [11]:
for cobraGroup in moduleCobras.values():
    pfi.homePhiSafe(cobraGroup, dir='ccw')

In [ ]:
c = pfiControl.PFI.allocateCobraList([(1,27)])
pfi.homePhiSafe(c, dir='ccw', iterations=20)

In [29]:
# move cobras to safe position
pfi.homePhiSafe(allCobras)
pfi.homeTheta(allCobras)
oddMoves = pfi.thetaToLocal(oddCobras, [np.deg2rad(90)]*len(oddCobras))
pfi.moveThetaPhi(oddCobras, oddMoves, np.zeros(len(oddCobras)))
evenMoves = pfi.thetaToLocal(evenCobras, [np.deg2rad(270)]*len(evenCobras))
pfi.moveThetaPhi(evenCobras, evenMoves, np.zeros(len(evenCobras)))

thetaSteps:  [ 891.39038766 3909.66746822 1148.81774277 1813.37364147 1995.80337869
  487.45555218 2022.24835946 4820.27210526 4295.7243541    26.68733938
 3560.27595624 5117.87229552 7886.48479961 4388.73574714 3101.52003765
 2964.17098193   73.48435231  127.49751775 2548.12967508 6277.80326879
  521.08035192 3146.12731656 3178.68946018 4810.72536261  103.47205905
  853.2445108  3339.38652207  505.27331008  283.83789456]
phiSteps:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
thetaSteps:  [4427.56542601 3362.0489191  4854.24666961 2856.86864008 4374.56854804
  430.48637271 3551.67663975 3766.9340677  2096.23299315  987.41522088
  462.68037727 4304.84857737  620.34042895 3769.56394171  215.14739244
  738.49488175 1587.81833939 4017.24710991 2126.61273647 4737.70290975
  308.8875733   734.39463398 3195.43122291 2032.94888421 2108.73681902
 1093.40938346 4645.08590019  913.4764013 ]
phiSteps:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [25]:
# for cobra measurement
steps = 200
sz = len(group1Cobras)
ar = np.zeros(sz)
for cobraGroup in moduleCobras.values():
    for i in range(25):
        pfi.moveSteps(cobraGroup, ar, ar+steps)
        time.sleep(0.1)
    for i in range(50):
        pfi.moveSteps(cobraGroup, ar+steps, ar)
        time.sleep(0.1)
    pfi.moveSteps(cobraGroup, ar+10000, ar+5000)
    for i in range(50):
        pfi.moveSteps(cobraGroup, ar-steps, ar)
        time.sleep(0.1)
    for i in range(25):
        pfi.moveSteps(cobraGroup, ar, ar-steps)
        time.sleep(0.1)
    pfi.moveSteps(cobraGroup, ar-10000, ar-5000)

In [5]:
len(oddCobras)

29

In [16]:
for cobraGroup in moduleCobras.values():
    pfi.moveAllThetaPhi(cobraGroup, np.deg2rad(180), 0.0)

thetaSteps:  [3841.65426635 3088.94954706 3378.94788463 3072.14902198 1952.815411
 2274.58841952 2765.61283922 3206.36751795 4984.22033894 2936.5169536
 2087.60459557 2693.25507136 3068.31041475 4321.12069111 2974.93368525
 2761.83902802 2799.07673357 2468.1391006  2425.96126887]
phiSteps:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
thetaSteps:  [3959.37521329 1595.00359291 2806.31560374 3829.75291852 2344.86993825
 1617.40330859 1818.57775577 2950.57358686 6208.38550056 1589.9221843
 4324.71876805 4391.29431787 2631.80244584 3794.44182991 1413.78682156
 3684.26223553 2128.46483413 2808.36138962 3368.1221625 ]
phiSteps:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
thetaSteps:  [ 2616.5442304   3661.71268882  3584.50256111  3035.32864495
  3556.2709541   3296.44432611  2436.12433233  4062.44738396
  2497.05603566  2869.38776853  2407.97600494  2175.98225307
  3258.40965019 10054.83912136  3307.46010079  3014.6118831
  4354.91693391  3896.84903523  2258.826

In [17]:
c1 = [pfi.cobraBySerial(2356), pfi.cobraBySerial(1)]

In [18]:
pfi.homePhiSafe(oneCobra)

In [19]:
pfi.moveAllSteps(mod1Cobras, -1000, 50)

In [20]:
pfi.reset()

In [23]:
pfi.homeTheta(oddCobras)

In [24]:
print(pfi.calibModel.motorFreq1, pfi.calibModel.motorFreq2)
print(pfi.calibModel.motorOntimeFwd1, pfi.calibModel.motorOntimeFwd2)
print(pfi.calibModel.tht0, pfi.calibModel.tht1)

[63241. 63241. 63745. 63492. 63241. 63492. 62500. 63492. 64000. 63492.
 62992. 63241. 62992. 63492. 64000. 63492. 63492. 62992. 63241. 64000.
 63745. 62992. 63492. 64000. 63241. 62500. 63241. 63241. 63492. 63492.
 63745. 63745. 63745. 63241. 63241. 64000. 62745. 64000. 62745. 62992.
 63492. 62500. 63241. 62256. 64000. 63492. 64000. 62992. 63492. 64516.
 64000. 64257. 64516. 63745. 64000. 64777. 63492.] [108108. 104575. 103896. 104575. 105263. 105960. 105960. 105263. 104575.
 105960. 107382. 105263. 105960. 105263. 105960. 103896. 105263. 106666.
 105263. 105960. 105960. 104575. 104575. 105263. 105960. 106666. 104575.
 108108. 105960. 105263. 108843. 106666. 105263. 105263. 106666. 105263.
 105960. 105263. 104575. 106666. 105960. 105263. 105960. 104575. 105960.
 105263. 105960. 105263. 106666. 106666. 105960. 105960. 104575. 104575.
 103896. 107382. 103896.]
[0.04572  0.035657 0.045208 0.034212 0.056261 0.035246 0.038806 0.04052
 0.036665 0.043863 0.034893 0.038601 0.045718 0.040192 0.0

In [30]:
evenMoves

array([3.76956797, 4.06529915, 4.15888868, 3.25191839, 4.9985311 ,
       0.4376001 , 4.41976045, 5.53224017, 1.76032219, 1.68508731,
       0.53446542, 3.4257414 , 0.3022908 , 4.35061223, 0.2357182 ,
       0.74528905, 1.90048232, 5.38378152, 2.59574531, 3.54750335,
       0.15997161, 1.71235326, 3.77197533, 2.11562347, 3.10330784,
       1.45843964, 3.60885824, 1.05639022])